In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlit.pyplot as plt
import pandas_profiling

# 数据概况

## 特征字段
|Field|Description|备注|
|--|--|--|
|SaleID|	交易ID|唯一编码|
|name|	汽车交易名称|已脱敏|
|regDate|	汽车注册日期|例如20160101，2016年01月01日|
|model|	车型编码|已脱敏|
|brand|	汽车品牌|已脱敏|
|bodyType|	车身类型|豪华轿车：0，微型车：1，厢型车：2，大巴车：3，敞篷车：4，双门汽车：5，商务车：6，搅拌车：7|
|fuelType|	燃油类型|汽油：0，柴油：1，液化石油气：2，天然气：3，混合动力：4，其他：5，电动：6|
|gearbox|	变速箱|手动：0，自动：1|
|power|	发动机功率|范围 [ 0, 600 ]|
|kilometer|	汽车已行驶公里|单位万km|
|notRepairedDamage|	汽车有尚未修复的损坏|是：0，否：1|
|regionCode|	地区编码|已脱敏|
|seller|	销售方|个体：0，非个体：|
|offerType|	报价类型|提供：0，请求：1|
|creatDate|	汽车上线时间|即开始售卖时间|
|**price**|二手车交易价格|（预测目标）|
|v0-14|	匿名特征|包含v0-14在内15个匿名特征|

## 评价指标
$$MAE=\frac{\sum_{i=1}^{n}|y_i-\hat{y}_i|}{n}$$

## 总结
本题是一个回归，特征有一些是数值特征、类型特征、时间特征。

# 数据准备